In [2]:
!python -m unittest tests.test_validator

EEEEFE.
ERROR: test_after_match_skip_validation (tests.test_validator.TestValidator.test_after_match_skip_validation)
Test validation of AFTER MATCH SKIP clause
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/tests/test_validator.py", line 139, in test_after_match_skip_validation
    errors, warnings = validate_match_recognize(ast)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/src/validator/match_recognize_validator.py", line 144, in validate_match_recognize
    return validator.validate(ast)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/src/validator/match_recognize_validator.py", line 87, in validate
    self._validate_cross_components(ast, pattern_vars)
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/src/validator/match_recognize_validator.py", 

In [2]:
!python -m unittest tests.test_parser

.FF
FAIL: test_navigation_expression (tests.test_parser.TestExpressionParser.test_navigation_expression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/tests/test_parser.py", line 13, in test_navigation_expression
    self.assertEqual(result["ast"].type, "navigation")
AssertionError: 'function' != 'navigation'
- function
+ navigation


FAIL: test_semantics_expression (tests.test_parser.TestExpressionParser.test_semantics_expression)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Row_match_recognize/tests/test_parser.py", line 20, in test_semantics_expression
    self.assertEqual(result["ast"].type, "navigation")
AssertionError: 'function' != 'navigation'
- function
+ navigation


----------------------------------------------------------------------
Ran 3 tests in 0.000s

F

In [4]:
!python -m unittest tests.test_ast

..
----------------------------------------------------------------------
Ran 2 tests in 0.000s

OK


In [10]:
!python   main.py

Generated AST for MATCH_RECOGNIZE clause:
MatchRecognizeAST(partition_by=[], order_by=[], measures=[], rows_per_match='ONE ROW PER MATCH', after_match_skip='SKIP PAST LAST ROW', pattern={}, subset={}, define=[], is_empty_match=False)

Parsed Expression AST:
binary
  navigation (nav: PREV, offset: 2) [RUNNING]
    navigation (nav: FIRST, offset: 3)
      identifier: A.totalprice
  literal: 5


In [4]:
!python -m unittest tests.test_expressions


DEBUG:transformations.expressions.expressions_parser:Parsed expression 'A + B' into AST: ExpressionAST(type='binary', value=None, operator='+', children=[ExpressionAST(type='identifier', value='A', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None), ExpressionAST(type='identifier', value='B', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None)], function_name=None, arguments=[], navigation_type=None, offset=None)
DEBUG:transformations.expressions.expressions_parser:Parsed expression '(A + B) * C' into AST: ExpressionAST(type='binary', value=None, operator='*', children=[ExpressionAST(type='parenthesized', value=None, operator=None, children=[ExpressionAST(type='binary', value=None, operator='+', children=[ExpressionAST(type='identifier', value='A', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None), ExpressionAST(type='identifier', value='B', operator

In [2]:
!python -m unittest tests.test_parser

E
ERROR: test_parser (unittest.loader._FailedTest.test_parser)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_parser
Traceback (most recent call last):
  File "/home/monierashraf/anaconda3/lib/python3.12/unittest/loader.py", line 137, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/tests/test_parser.py", line 3, in <module>
    from transformations.parser import build_enhanced_match_recognize_ast
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/__init__.py", line 5, in <module>
    from .automaton import (
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/__init__.py", line 5, in <module>
    from .automaton_builder import AutomatonBuilder
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_bu

In [1]:
!python -m unittest tests.test_pattern


E
ERROR: test_pattern (unittest.loader._FailedTest.test_pattern)
----------------------------------------------------------------------
ImportError: Failed to import test module: test_pattern
Traceback (most recent call last):
  File "/home/monierashraf/anaconda3/lib/python3.12/unittest/loader.py", line 137, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/tests/test_pattern.py", line 9, in <module>
    from transformations.pattern.pattern_ast import PatternAST
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/__init__.py", line 5, in <module>
    from .automaton import (
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/__init__.py", line 9, in <module>
    from .automaton_builder import AutomatonBuilder
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_builder.p

In [6]:
!python -m unittest tests.test_pattern


DEBUG:transformations.pattern.pattern_parser:Parsed pattern 'A B+ C?' into AST: PatternAST(type='concatenation', value=None, quantifier=None, quantifier_min=None, quantifier_max=None, children=[PatternAST(type='literal', value='A', quantifier=None, quantifier_min=None, quantifier_max=None, children=[], excluded=False), PatternAST(type='quantifier', value=None, quantifier='+', quantifier_min=None, quantifier_max=None, children=[PatternAST(type='literal', value='B', quantifier=None, quantifier_min=None, quantifier_max=None, children=[], excluded=False)], excluded=False), PatternAST(type='quantifier', value=None, quantifier='?', quantifier_min=None, quantifier_max=None, children=[PatternAST(type='literal', value='C', quantifier=None, quantifier_min=None, quantifier_max=None, children=[], excluded=False)], excluded=False)], excluded=False)
.DEBUG:transformations.pattern.pattern_parser:Parsed pattern 'A (B|C)+ D?' into AST: PatternAST(type='concatenation', value=None, quantifier=None, quant

In [3]:
!python -m unittest tests.test_type_inference

.........F..F............Unknown identifier: unknown_column
.F..
FAIL: test_invalid_arithmetic_operands (tests.test_type_inference.TestTypeInference.test_invalid_arithmetic_operands)
Test type inference for arithmetic operators with invalid operands.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/tests/test_type_inference.py", line 236, in test_invalid_arithmetic_operands
    with self.assertRaises(TypeCheckError):
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AssertionError: TypeCheckError not raised

FAIL: test_invalid_navigation_functions (tests.test_type_inference.TestTypeInference.test_invalid_navigation_functions)
Test type inference for invalid navigation functions.
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/tests/test_type_inference.py",

In [5]:
!python -m unittest tests.test_comprehensive


.Unknown expression type: function
E.E.EEEE..FEEE
ERROR: test_function_validation (tests.test_comprehensive.TestASTParsingAndValidation.test_function_validation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/tests/test_comprehensive.py", line 84, in test_function_validation
    AutomatonBuilder(ast, self.var_conditions)
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_builder.py", line 32, in __init__
    self._validate_types()
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_builder.py", line 72, in _validate_types
    errors = validate_pattern_variable_conditions(self.variable_conditions, self.schema)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/monierashraf/Desktop/llm/Match_recognize/project/transformation

In [30]:
!python -m unittest tests.test_system

DEBUG:transformations.parser.builder:Entering MATCH_RECOGNIZE clause (enhanced AST builder)
DEBUG:transformations.parser.builder:Extracted PARTITION BY: ['custkey']
DEBUG:transformations.parser.builder:Extracted ORDER BY: ['orderdate']
DEBUG:transformations.expressions.expressions_parser:Parsed expression 'FIRST(A.price)' into AST: ExpressionAST(type='navigation', value=None, operator=None, children=[], function_name=None, arguments=[ExpressionAST(type='qualified_identifier', value='A.price', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None), ExpressionAST(type='literal', value='0', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None)], navigation_type='FIRST', offset=None)
DEBUG:transformations.expressions.expressions_parser:Parsed expression 'LAST(B.price)' into AST: ExpressionAST(type='navigation', value=None, operator=None, children=[], function_name=None, arguments=[ExpressionAST(type='quali

In [1]:
!python -m unittest tests.test_system

DEBUG:transformations.parser.builder:Entering MATCH_RECOGNIZE clause (enhanced AST builder)
DEBUG:transformations.parser.builder:Extracted PARTITION BY: ['custkey']
DEBUG:transformations.parser.builder:Extracted ORDER BY: ['orderdate']
DEBUG:transformations.expressions.expressions_parser:Parsed expression 'FIRST(A.price)' into AST: ExpressionAST(type='navigation', value=None, operator=None, children=[], function_name=None, arguments=[ExpressionAST(type='qualified_identifier', value='A.price', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None), ExpressionAST(type='literal', value='0', operator=None, children=[], function_name=None, arguments=[], navigation_type=None, offset=None)], navigation_type='FIRST', offset=None)
DEBUG:transformations.expressions.expressions_parser:Parsed expression 'LAST(B.price)' into AST: ExpressionAST(type='navigation', value=None, operator=None, children=[], function_name=None, arguments=[ExpressionAST(type='quali

In [11]:
!python -m unittest engine.run_tests

E
ERROR: engine (unittest.loader._FailedTest.engine)
----------------------------------------------------------------------
ImportError: Failed to import test module: engine
Traceback (most recent call last):
  File "/home/monierashraf/anaconda3/lib/python3.12/unittest/loader.py", line 137, in loadTestsFromName
    module = __import__(module_name)
             ^^^^^^^^^^^^^^^^^^^^^^^
ModuleNotFoundError: No module named 'engine'


----------------------------------------------------------------------
Ran 1 test in 0.000s

FAILED (errors=1)


In [14]:
!python -m unittest tests.test_comprehensive

...............
----------------------------------------------------------------------
Ran 15 tests in 0.001s

OK


In [1]:
!find . -type f -name "*.py" | sort

./1.py
./2.py
./3.py
./maiaas22222n.py
./main.py
./src/ast/ast_builder.py
./src/ast/expression_ast.py
./src/ast/__init__.py
./src/ast/match_recognize_ast.py
./src/ast/pattern_ast.py
./src/ast/pattern_optimizer.py
./src/evaluator/evaluation_engine.py
./src/grammar/__init__.py
./src/grammar/TrinoLexer.py
./src/grammar/TrinoParserListener.py
./src/grammar/TrinoParser.py
./src/grammar/TrinoParserVisitor.py
./src/__init_.py
./src/parser/antlr_parser.py
./src/parser/expression_parser.py
./src/parser/__init__.py
./src/parser/pattern_parser.py
./src/validator/__init__.py
./src/validator/match_recognize_validator.py
./src/validator/semantic_validator.py
./tests/__init__.py
./tests/test_ast.py
./tests/test_parser.py
./tests/test_validator.py


In [4]:
import sys
import pandas as pd
# Use an absolute import for match_recognize.
from transformations.match_recognize import match_recognize

query = """
    SELECT id, name FROM employees MATCH_RECOGNIZE (
        PARTITION BY department, region
        ORDER BY hire_date
        MEASURES salary AS avg_salary
        PATTERN (A)
        DEFINE A AS salary > 1000
    );
    """
    
data = [
        {"id": 1, "name": "Alice",   "department": "Sales", "region": "West", "hire_date": "2021-01-01", "salary": 1200},
        {"id": 2, "name": "Bob",     "department": "Sales", "region": "West", "hire_date": "2021-01-02", "salary": 1300},
        {"id": 3, "name": "Charlie", "department": "Sales", "region": "West", "hire_date": "2021-01-03", "salary": 900},
        {"id": 4, "name": "Diana",   "department": "Sales", "region": "West", "hire_date": "2021-01-04", "salary": 1100},
    ]
    
try:
        output_df = match_recognize(query, pd.DataFrame(data))
        print("Match Recognize Output:")
        print(output_df)
except Exception as e:
        print(f"Error: {str(e)}")

ImportError: cannot import name 'AutomatonBuilder' from partially initialized module 'transformations.automaton.automaton_builder' (most likely due to a circular import) (/home/monierashraf/Desktop/llm/Match_recognize/project/transformations/automaton/automaton_builder.py)